# Project BBT045
Question:
> How pharmaceutical pollution affect microbial community composition

In [7]:
#Check that we have access to the files in the common directory:
with open("/cephyr/NOBACKUP/groups/bbt045_2024/project_group1/filelist.txt") as f: # The with keyword automatically closes the file when you are done
    print(f.read())
    #ref https://stackoverflow.com/questions/18256363/how-do-i-print-the-content-of-a-txt-file-in-python

#data located at /cephyr/NOBACKUP/groups/bbt045_2024/PROJECT_DATA
downstream_1.5km_2012_1.fastq.gz
downstream_1.5km_2012_2.fastq.gz
downstream_17.5km_2011_1.fastq.gz
downstream_17.5km_2011_2.fastq.gz
downstream_2.3km_2011_1.fastq.gz
downstream_2.3km_2011_2.fastq.gz
downstream_2.7km_2011_1.fastq.gz
downstream_2.7km_2011_2.fastq.gz
effluent_point_2011_1.fastq.gz
effluent_point_2011_2.fastq.gz
upstream_1.9km_2011_1.fastq.gz
upstream_1.9km_2011_2.fastq.gz
upstream_150m_2012_1.fastq.gz
upstream_150m_2012_2.fastq.gz
upstream_2.2km_2011_1.fastq.gz
upstream_2.2km_2011_2.fastq.gz

